In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('online_shoppers_intention.csv')
data.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [3]:
data.shape

(12330, 18)

In [34]:
# Checking for NaN Values
data.isna().sum()

administrative             0
administrative_duration    0
informational              0
informational_duration     0
productrelated             0
productrelated_duration    0
bouncerates                0
exitrates                  0
pagevalues                 0
specialday                 0
month                      0
operatingsystems           0
browser                    0
region                     0
traffictype                0
visitortype                0
weekend                    0
revenue                    0
dtype: int64

In [4]:
col_l = []
for col in data.columns:
    col = col.replace(' ','_')
    col_l.append(col.lower())
data.columns = col_l
print(data.columns)

Index(['administrative', 'administrative_duration', 'informational',
       'informational_duration', 'productrelated', 'productrelated_duration',
       'bouncerates', 'exitrates', 'pagevalues', 'specialday', 'month',
       'operatingsystems', 'browser', 'region', 'traffictype', 'visitortype',
       'weekend', 'revenue'],
      dtype='object')


In [5]:
data['month'].value_counts()

May     3364
Nov     2998
Mar     1907
Dec     1727
Oct      549
Sep      448
Aug      433
Jul      432
June     288
Feb      184
Name: month, dtype: int64

In [6]:
data['revenue'].value_counts()

False    10422
True      1908
Name: revenue, dtype: int64

In [7]:
# Data for customers that did not convert - did not buy items in the store
data_false = data[data['revenue'] == False]
data_false.shape

(10422, 18)

In [8]:
# Data for customers that converted - spent in the store
data_true = data[data['revenue'] == True]
data_true.shape

(1908, 18)

In [36]:
# PERCENTAGE OF RETURNING VISITORS THAT CONVERTED - SPENT MONEY
percent_buy = len(data_true)/len(data)
print("Ratio of visitors that bought items ", round(percent_buy,2))

Percentage of visitors that bought items  0.15


In [11]:
# Use data where 'Revenue' is True
data_true['visitortype'].value_counts()

Returning_Visitor    1470
New_Visitor           422
Other                  16
Name: visitortype, dtype: int64

In [12]:
# Export dataframe where 'revenue' is True for use in Tableau
data_true.to_excel("E:\\SKUL\\PROGRAMMING_SKULS\\ironhack\\COURSE\\Mid-Term-Project\\Data\\Customer_with_revenue.xlsx")

In [47]:
# Mothly Breakdown of customers that bought items 
data_true['month'].value_counts()

Nov     760
May     365
Dec     216
Mar     192
Oct     115
Sep      86
Aug      76
Jul      66
June     29
Feb       3
Name: month, dtype: int64

In [38]:
# Percentage of revenue from returning visitors
perc_return_customer_revenue = len(data_true[data_true['visitortype'] == 'Returning_Visitor'])/len(data_true)
print("Ratio of returning customers that buy items: ", round(perc_return_customer_revenue, 2))

Percentage of returning customers that buy items:  0.77


In [39]:
# percentage of revenue from new visitors
perc_revenue_new_customer = len(data_true[data_true['visitortype'] == 'New_Visitor'])/len(data_true)

print("Ratio of new customers that buy items: ", round(perc_revenue_new_customer, 2))

Percentage of new customers that buy items:  0.22


In [40]:
# Percentage of returning visitors that didnot buy
# We use the dataframe where 'revenue' is False
no_revenue_return_customer = len(data_false[data_false['visitortype'] == 'Returning_Visitor'])/len(data_false)
round(no_revenue_return_customer, 2)

0.87

In [30]:
# This percentage could be that returning customers simply make up a huge chunk of our website visitors
overall_return = len(data[data['visitortype'] == 'Returning_Visitor'])/len(data)
print("Returning Customers Percentage: ", round(overall_return,2))

Returning Customers Percentage:  0.86


In [17]:
# from the above calculation, 85.5% of visitors are returning customers

In [28]:
# New customers
overall_new = len(data[data['visitortype'] == 'New_Visitor'])/len(data)
print("New Customers Percentage: ", round(overall_new,2))

New Customers Percentage:  0.14


In [18]:
# Revenue on special days
special_days = data[data['specialday'] == 1]
special_days.head()

,administrative,administrative_duration,informational,informational_duration,productrelated,productrelated_duration,bouncerates,exitrates,pagevalues,specialday,month,operatingsystems,browser,region,traffictype,visitortype,weekend,revenue
20,0,0.0,0,0.0,8,136.166667,0.0,0.008333,0.000000,1.0,Feb,2,2,5,1,Returning_Visitor,True,False
52,0,0.0,0,0.0,2,29.000000,0.0,0.100000,0.000000,1.0,Feb,2,4,4,2,Returning_Visitor,True,False
78,0,0.0,0,0.0,1,0.000000,0.2,0.200000,0.000000,1.0,Feb,1,1,1,3,Returning_Visitor,True,False
101,4,61.0,0,0.0,19,607.000000,0.0,0.026984,17.535959,1.0,Feb,1,1,7,4,Returning_Visitor,True,True
174,5,41.3,0,0.0,24,446.927778,0.0,0.008602,0.000000,1.0,Feb,2,2,8,1,Returning_Visitor,True,False


In [44]:
percentage_of_special_day = len(special_days)/len(data)
print("Ratio of visitors on special days: ",round(percentage_of_special_day,2) )

Percentage of visitors on special days:  0.01


In [35]:
special_revenue_perc = len(special_days[special_days['revenue'] == True])/len(special_days)
print("Ratio of revenue customers on special days: ",round(special_revenue_perc,2))

Percentage of revenue customers on special days:  0.06


In [ ]:
# Only 6% of visitors on special day converted